In [2]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO



In [14]:
q = '''
SELECT to_char(budget.start_at, 'yyyy-mm-dd') AS ad_date,
       bid.keyword,
    max(bid.bid_price) AS max_bid_price,
 count(DISTINCT product.id) AS bid_product_cnt
FROM service1_quicket.ad_set_daily_budget budget
JOIN service1_quicket.ad_set_product product ON budget.ad_set_id = product.ad_set_id
JOIN service1_quicket.ad_set_product_bid_keyword bid ON bid.ad_set_product_id = product.id
WHERE budget.start_at BETWEEN '2021-03-30' AND '2021-04-03'
GROUP BY 1,
         2
'''

df = pd.read_sql(q, con=bun_dw)

In [20]:
cnt = df.groupby('keyword', as_index=False)['bid_product_cnt'].mean()
cnt.columns = ['keyword', 'mean_p_cnt']


In [21]:
cnt.keyword.nunique()

19485

In [22]:
keywords = cnt.sort_values('mean_p_cnt', ascending=False)[:1000]['keyword']

In [23]:
df[df.keyword.isin(keywords)].sort_values(['keyword', 'max_bid_price']).to_csv('df.csv', index=False)

In [24]:
tmp = df[df.keyword.isin(keywords)].sort_values(['keyword', 'max_bid_price'])
tmp.head()

,ad_date,keyword,max_bid_price,bid_product_cnt
22,2021-04-03,.,50,96
18,2021-03-30,.,55,107
19,2021-03-31,.,55,97
20,2021-04-01,.,55,97
21,2021-04-02,.,55,97


In [27]:
df.groupby('ad_date')[['max_bid_price', 'bid_product_cnt']].describe()

max_bid_price                                                 \
                   count       mean         std   min   25%   50%   75%   
ad_date                                                                   
2021-03-30       13338.0  76.294047   98.992190  50.0  50.0  50.0  60.0   
2021-03-31       13015.0  75.753746  101.233500  50.0  50.0  50.0  55.0   
2021-04-01       14772.0  75.938262   99.076501  50.0  50.0  50.0  60.0   
2021-04-02       14288.0  76.122270   96.058160  50.0  50.0  50.0  60.0   
2021-04-03       11849.0  79.497848  101.561165  50.0  50.0  50.0  60.0   

                   bid_product_cnt                                          \
               max           count      mean       std  min  25%  50%  75%   
ad_date                                                                      
2021-03-30  1000.0         13338.0  2.496101  6.583806  1.0  1.0  1.0  2.0   
2021-03-31  1000.0         13015.0  2.597311  7.026670  1.0  1.0  1.0  2.0   
2021-04-01  1000.0         14772.0  2.430680  6.345010  1.0  1.0  1.0  2.0   
2021-04-02  1000.0         14288.0  2.640957  6.478149  1.0  1.0  1.0  2.0   
2021-04-03  1000.0         11849.0  2.250063  5.615208  1.0  1.0  1.0  2.0   

                   
              max  
ad_date            
2021-03-30  207.0  
2021-03-31  244.0  
2021-04-01  203.0  
2021-04-02  177.0  
2021-04-03  166.0

In [28]:
tmp.groupby('ad_date')[['max_bid_price', 'bid_product_cnt']].describe()

max_bid_price                                                    \
                   count        mean         std   min   25%    50%    75%   
ad_date                                                                      
2021-03-30         951.0  237.691903  248.564952  50.0  65.0  120.0  305.0   
2021-03-31         957.0  243.803553  253.379470  50.0  70.0  125.0  310.0   
2021-04-01         967.0  250.879007  252.182713  50.0  70.0  140.0  315.0   
2021-04-02         979.0  243.299285  243.418151  50.0  70.0  130.0  310.0   
2021-04-03         937.0  243.719317  240.189925  50.0  70.0  130.0  315.0   

                   bid_product_cnt                                        \
               max           count       mean        std  min  25%   50%   
ad_date                                                                    
2021-03-30  1000.0           951.0  15.948475  20.046065  1.0  6.0  10.0   
2021-03-31  1000.0           957.0  16.779519  21.060842  1.0  7.0  10.0   
2021-04-01  1000.0           967.0  16.423992  19.866015  1.0  7.0  10.0   
2021-04-02  1000.0           979.0  16.774259  19.641863  1.0  7.0  10.0   
2021-04-03  1000.0           937.0  13.098186  16.277474  1.0  5.0   8.0   

                         
             75%    max  
ad_date                  
2021-03-30  17.0  207.0  
2021-03-31  17.0  244.0  
2021-04-01  17.0  203.0  
2021-04-02  17.0  177.0  
2021-04-03  14.0  166.0

In [37]:
tmp.sort_values(['keyword', 'ad_date']).to_csv('df.csv', index=False)

In [29]:
q = '''
SELECT imp.ad_date AS imp_date,
       bid.keyword,
       max(bid.bid_price) AS max_bid_price,
       count(DISTINCT bid.ad_set_product_id) AS bid_product_cnt
FROM
  (SELECT YEAR||MONTH||DAY AS ad_date,
                              ad_set_product_id
   FROM bun_log_db.api_event_type_impression_ad
   WHERE YEAR||MONTH||DAY BETWEEN '20210330' AND '20210403') imp
JOIN service1_quicket.ad_set_product_bid_keyword bid ON bid.ad_set_product_id = imp.ad_set_product_id
GROUP BY 1,
         2
'''
imp = pd.read_sql(q, con=bun_dw)

In [31]:
cnt2 = imp.groupby('keyword', as_index=False)['bid_product_cnt'].mean()
cnt2.columns = ['keyword', 'mean_p_cnt']


In [32]:
keywords2 = cnt2.sort_values('mean_p_cnt', ascending=False)[:1000]['keyword']

In [33]:

tmp2 = imp[imp.keyword.isin(keywords2)].sort_values(['keyword', 'max_bid_price'])

In [35]:
tmp2.groupby('imp_date')[['max_bid_price', 'bid_product_cnt']].describe()

max_bid_price                                                     \
                 count        mean         std   min   25%    50%     75%   
imp_date                                                                    
20210330         987.0  260.775076  257.076582  50.0  75.0  150.0  325.00   
20210331         986.0  264.685598  259.723306  50.0  80.0  150.0  330.00   
20210401         992.0  274.692540  258.037852  50.0  80.0  165.0  376.25   
20210402         988.0  268.051619  252.797969  50.0  80.0  160.0  366.25   
20210403         989.0  273.559151  256.611901  50.0  85.0  160.0  375.00   

                 bid_product_cnt                                               \
             max           count       mean        std  min   25%   50%   75%   
imp_date                                                                        
20210330  1000.0           987.0  30.077001  35.146811  1.0  12.0  18.0  31.0   
20210331  1000.0           986.0  30.907708  35.613444  1.0  13.0  18.0  32.0   
20210401  1000.0           992.0  31.518145  36.106772  5.0  13.0  19.0  33.0   
20210402  1000.0           988.0  30.557692  34.507531  4.0  13.0  18.0  32.0   
20210403  1000.0           989.0  30.435794  34.399434  4.0  12.0  18.0  33.0   

                 
            max  
imp_date         
20210330  313.0  
20210331  345.0  
20210401  344.0  
20210402  282.0  
20210403  291.0

In [36]:
imp.groupby('imp_date')[['max_bid_price', 'bid_product_cnt']].describe()

max_bid_price                                                        \
                 count       mean        std   min   25%   50%   75%     max   
imp_date                                                                       
20210330       18464.0  73.991280  93.999847  50.0  50.0  50.0  60.0  1000.0   
20210331       18629.0  74.597402  99.047829  50.0  50.0  50.0  55.0  1000.0   
20210401       19909.0  74.709679  97.040565  50.0  50.0  50.0  60.0  1000.0   
20210402       19494.0  74.039192  93.122226  50.0  50.0  50.0  60.0  1000.0   
20210403       18843.0  75.528578  97.782082  50.0  50.0  50.0  60.0  1000.0   

         bid_product_cnt                                                  
                   count      mean        std  min  25%  50%  75%    max  
imp_date                                                                  
20210330         18464.0  3.413074  10.432645  1.0  1.0  1.0  2.0  313.0  
20210331         18629.0  3.448655  10.585547  1.0  1.0  1.0  2.0  345.0  
20210401         19909.0  3.376262  10.446512  1.0  1.0  1.0  2.0  344.0  
20210402         19494.0  3.439520  10.112643  1.0  1.0  1.0  2.0  282.0  
20210403         18843.0  3.411665  10.262590  1.0  1.0  1.0  2.0  291.0

In [3]:
q = '''
SELECT a.ad_date,
       a.ad_set_product_id,
       a.max_bid_price,
       b.keyword
FROM
  (SELECT to_char(budget.start_at, 'yyyy-mm-dd') AS ad_date,
          product.id AS ad_set_product_id,
          max(bid.bid_price) AS max_bid_price
   FROM service1_quicket.ad_set_daily_budget budget
   JOIN service1_quicket.ad_set_product product ON budget.ad_set_id = product.ad_set_id
   JOIN service1_quicket.ad_set_product_bid_keyword bid ON bid.ad_set_product_id = product.id
   WHERE budget.start_at BETWEEN '2021-03-30' AND '2021-04-03'
   GROUP BY 1,
            2) a
JOIN
  (SELECT product.id AS ad_set_product_id,
          bid.keyword,
          bid.bid_price
   FROM service1_quicket.ad_set_daily_budget budget
   JOIN service1_quicket.ad_set_product product ON budget.ad_set_id = product.ad_set_id
   JOIN service1_quicket.ad_set_product_bid_keyword bid ON bid.ad_set_product_id = product.id
   WHERE budget.start_at BETWEEN '2021-03-30' AND '2021-04-03') b ON a.ad_set_product_id = b.ad_set_product_id
AND a.max_bid_price = b.bid_price
GROUP BY 1,
         2,
         3,
         4
'''
d = pd.read_csv('ad_df.csv')

In [8]:
cnt = d.groupby('ad_set_product_id', as_index=False)['ad_date'].nunique()
cnt

,ad_set_product_id,ad_date
0,179254,1
1,179651,1
2,180778,2
3,180982,4
4,180983,4
...,...,...
10439,236822,2
10440,237384,2
10441,240725,5
10442,241067,5


In [6]:
d.head()

,ad_date,ad_set_product_id,max_bid_price,keyword
0,2021-04-02,233935,100,탈모완화샴푸
1,2021-03-30,235021,345,김치냉장고
2,2021-04-01,235021,345,김치냉장고
3,2021-03-30,235652,270,모니터
4,2021-03-31,235652,270,모니터


In [7]:
d.ad_date.unique()

array(['2021-04-02', '2021-03-30', '2021-04-01', '2021-03-31',
       '2021-04-03'], dtype=object)

In [16]:
tmp = d[d['ad_set_product_id'].isin(cnt[cnt['ad_date'] >= 3].ad_set_product_id)]

In [15]:
df

,ad_date,keyword,max_bid_price,bid_product_cnt
0,2021-04-03,돌체,60,3
1,2021-04-01,아우디,95,5
2,2021-04-03,아우디,95,3
3,2021-04-03,bmw,155,8
4,2021-04-02,포토샵,325,10
...,...,...,...,...
67345,2021-04-03,다이어트카페,100,1
67346,2021-04-02,와플기,100,1
67347,2021-04-02,수입도자기,250,1
67348,2021-04-02,골든구스스니커즈,55,2


In [22]:
m = pd.merge(tmp, df, on=['ad_date', 'keyword']).sort_values(['ad_set_product_id', 'ad_date'])
m = m.iloc[:, :5]
m.columns = ['ad_date', 'ad_set_product_id', 'ad_set_product_id_max_bid_price', 'keyword', 'keyword_max_bid_price']

In [26]:
m.sort_values(['ad_set_product_id', 'ad_date', 'keyword']).to_csv('compare.csv', index=False)

In [29]:
d.groupby('ad_date')['max_bid_price'].describe()

,count,mean,std,min,25%,50%,75%,max
ad_date,,,,,,,,
2021-03-30,11503.0,170.980614,223.357670,50.0,50.0,60.0,150.0,1000.0
2021-03-31,11902.0,173.588052,225.602992,50.0,50.0,55.0,160.0,1000.0
2021-04-01,13997.0,151.569265,210.937327,50.0,50.0,60.0,110.0,1000.0
2021-04-02,15921.0,135.236480,191.509129,50.0,50.0,60.0,100.0,1000.0
2021-04-03,10984.0,139.363165,196.077712,50.0,50.0,60.0,100.0,1000.0


In [53]:
t = d.groupby(['ad_date', 'ad_set_product_id'], as_index=False)['max_bid_price'].mean()
t.groupby('ad_date')['max_bid_price'].describe()

,count,mean,std,min,25%,50%,75%,max
ad_date,,,,,,,,
2021-03-30,5456.0,268.672104,268.879368,50.0,60.0,125.0,411.25,1000.0
2021-03-31,5746.0,268.518970,268.967014,50.0,55.0,125.0,395.00,1000.0
2021-04-01,5813.0,253.677103,268.689219,50.0,60.0,100.0,370.00,1000.0
2021-04-02,5959.0,236.027018,259.325456,50.0,60.0,100.0,350.00,1000.0
2021-04-03,4280.0,236.349299,258.913401,50.0,55.0,100.0,345.00,1000.0


In [55]:
d.sort_values('max_bid_price', ascending=False)

,ad_date,ad_set_product_id,max_bid_price,keyword
19181,2021-03-30,221884,1000,원피스
29135,2021-04-02,223340,1000,라이젠
29348,2021-03-31,220419,1000,문상
15046,2021-03-30,220843,1000,샤넬
5894,2021-04-01,218491,1000,맥북
...,...,...,...,...
46884,2021-03-30,220268,50,타로상담
7382,2021-04-03,223368,50,해산물
26193,2021-03-30,219488,50,Iphone12promax
46881,2021-03-31,220235,50,큐베이스


In [30]:
df.groupby('ad_date')['max_bid_price'].describe()

,count,mean,std,min,25%,50%,75%,max
ad_date,,,,,,,,
2021-03-30,13341.0,76.318117,98.996443,50.0,50.0,50.0,60.0,1000.0
2021-03-31,13018.0,75.776617,101.237142,50.0,50.0,50.0,55.0,1000.0
2021-04-01,14795.0,75.916188,98.988925,50.0,50.0,50.0,60.0,1000.0
2021-04-02,14317.0,76.114060,95.951610,50.0,50.0,50.0,60.0,1000.0
2021-04-03,11879.0,79.479754,101.422607,50.0,50.0,50.0,60.0,1000.0


In [40]:
a = d[(d['ad_date'] == '2021-03-31')].groupby('ad_set_product_id', as_index=False)['max_bid_price'].mean()
b = d[(d['ad_date'] == '2021-04-01')].groupby('ad_set_product_id', as_index=False)['max_bid_price'].mean()

In [41]:
a.ad_set_product_id = a.ad_set_product_id.astype(str)
b.ad_set_product_id = b.ad_set_product_id.astype(str)

In [43]:
pd.merge(a, b, on='ad_set_product_id').to_csv('0331vs0401.csv', index=False)

In [44]:
c = pd.merge(a, b, on='ad_set_product_id')

In [48]:
c[c['max_bid_price_x'] == c['max_bid_price_y']]

,ad_set_product_id,max_bid_price_x,max_bid_price_y
0,180982,155,155
1,180983,85,85
2,180984,55,55
3,180985,70,70
4,180986,95,95
...,...,...,...
4180,235653,270,270
4181,235654,50,50
4182,240725,50,50
4183,241067,50,50
